Uses FastText (https://arxiv.org/abs/1607.01759) in Keras to predict sentiment of tweets.

In [35]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.datasets import imdb

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [36]:
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}

    >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.

    Example: adding bi-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    >>> add_ngram(sequences, token_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]

    Example: adding tri-gram
    >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences

In [37]:
# Set parameters:
# ngram_range = 2 will add bi-grams features
ngram_range = 1
max_features = 80000
maxlen = 150
batch_size = 32
embedding_dims = 50
epochs = 5

In [38]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(
    np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(
    np.mean(list(map(len, x_test)), dtype=int)))

np.load = np_load_old

Loading data...
25000 train sequences
25000 test sequences
Average train sequence length: 238
Average test sequence length: 230


In [39]:
if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting x_train and x_test with n-grams features
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(
        np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(
        np.mean(list(map(len, x_test)), dtype=int)))

In [40]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 150)
x_test shape: (25000, 150)


In [41]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

# we add a GlobalAveragePooling1D, which will average the embeddings
# of all words in the document
model.add(GlobalAveragePooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [42]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - ETA: 8:21 - loss: 0.6934 - acc: 0.468 - ETA: 3:02 - loss: 0.6933 - acc: 0.468 - ETA: 1:59 - loss: 0.6933 - acc: 0.462 - ETA: 1:32 - loss: 0.6931 - acc: 0.495 - ETA: 1:16 - loss: 0.6930 - acc: 0.500 - ETA: 1:07 - loss: 0.6928 - acc: 0.536 - ETA: 1:00 - loss: 0.6926 - acc: 0.552 - ETA: 55s - loss: 0.6926 - acc: 0.556 - ETA: 51s - loss: 0.6924 - acc: 0.58 - ETA: 48s - loss: 0.6922 - acc: 0.59 - ETA: 46s - loss: 0.6921 - acc: 0.59 - ETA: 44s - loss: 0.6920 - acc: 0.60 - ETA: 42s - loss: 0.6918 - acc: 0.61 - ETA: 41s - loss: 0.6917 - acc: 0.62 - ETA: 39s - loss: 0.6917 - acc: 0.62 - ETA: 38s - loss: 0.6915 - acc: 0.62 - ETA: 37s - loss: 0.6914 - acc: 0.63 - ETA: 36s - loss: 0.6911 - acc: 0.64 - ETA: 35s - loss: 0.6910 - acc: 0.64 - ETA: 35s - loss: 0.6910 - acc: 0.64 - ETA: 34s - loss: 0.6908 - acc: 0.65 - ETA: 33s - loss: 0.6906 - acc: 0.65 - ETA: 33s - loss: 0.6904 - acc: 0.66 - ETA:

In [43]:
model.save('FastText_Unigram_150len80kfeatures.h5')

In [44]:
from keras.models import load_model
from keras.datasets import imdb
from keras.preprocessing import sequence
import spacy 
import numpy as np

nlp = spacy.load('en_core_web_sm')

# Set parameters:
# ngram_range = 2 will add bi-grams features
ngram_range = 1
max_features = 80000
maxlen = 150
batch_size = 32
embedding_dims = 50
epochs = 5

# load model
model = load_model('FastText_Unigram_150len80kfeatures.h5')
# summarize model.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 50)           4000000   
_________________________________________________________________
global_average_pooling1d_2 ( (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 4,000,051
Trainable params: 4,000,051
Non-trainable params: 0
_________________________________________________________________


In [45]:
word_index = imdb.get_word_index()

In [99]:
doc = "This is a really happy tweet! I am so happy wonderful marvellous absolutely amazing!" 

nlp_doc = nlp(doc)
indices_list = []
for token in nlp_doc:
    token_text = token.text.lower()
    indices_list.append(word_index[token_text] if token_text in word_index else 0)

print(indices_list)

[11, 6, 3, 63, 651, 0, 0, 10, 241, 35, 651, 386, 9844, 424, 477, 0]


In [100]:
def sentence_to_indices(sentence):
    nlp_doc = nlp(sentence)
    indices_list = []
    for token in nlp_doc:
        token_text = token.text.lower()
        indices_list.append(word_index[token_text] if token_text in word_index else 0)
    return(indices_list)
x_sample = sentence_to_indices(doc)
x_sample

[11, 6, 3, 63, 651, 0, 0, 10, 241, 35, 651, 386, 9844, 424, 477, 0]

In [101]:
test_sample = sequence.pad_sequences([x_sample], maxlen = maxlen, padding = 'post')
test_sample

array([[  11,    6,    3,   63,  651,    0,    0,   10,  241,   35,  651,
         386, 9844,  424,  477,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [102]:
model.predict(test_sample)

array([[0.6108945]], dtype=float32)

In [103]:
len(word_index)

88584

In [104]:
def fast_text_sentiment(prediction):
    score = (prediction - 0.5) * 2
    return score 

In [105]:
print(fast_text_sentiment(model.predict(test_sample)))

[[0.221789]]
